## REGEX 101
[Here](https://www.rexegg.com/) is a great resource for anyone that wants to did further into the black hole that is regular expressions..
And [here](https://pythex.org) is a great site for testing out your regular expressions.. 
Esp dealing with python.

And [here](https://docs.python.org/3.4/library/re.html), of course, is the actual documentation.

Rather than teaching all the ways to use regex.. I will bring in some test samples to try and match!

### Don't worry about these too much. Just know they are helping!

In [1]:
from utils import (
    findall, 
    match,
    search,
    check_matches)

First.. 
Let's look at some different ways you can go about matching.

In [2]:
text = "Here is some sample 1 text with 2 numbers 3!"
r = r"\d"

# Notice that you can iterate over the answers with this one.
print('findall..')
for val in findall(r, text):
    print(val)
    
   
# For the next two.. You capture the match, then test to see if it exists.
print('\nmatch..')
m = match(r, text)
print(m.group() if m else None)

print('\nsearch..')
m = search(r, text)
print(m.group() if m else None)

findall..
1
2
3

match..
None

search..
1


### With python.. You have the ability to add in comments to your regex to make them easier to read.

In [3]:
# Let's try to find one for match just for fun.. Remember that we need to start at the beginning of the string!
match(r"""
      [\w\s]+             # match all words or spaces
      (?P<digit>[0-9])    # group the LAST digit, and give it a name
      .*                  # everything after (!)""", 
      text)\
    .group('digit')

# NOTE: It stretched as far as it could before returning the number.. 

'3'

General syntax is as follows: 

```python
pattern = re.compile(r"<regex_pattern>")   # define your pattern, optionally pass in flags!
matches = re.<match_func>(pattern, string)

# validate here to see if you found anything with an if statement. 
# if you found something.. the match will be "truthy"
if matches:
    print(matches.group(<name_of_group>))
```

Now you try!!!

### Extract `12 years old`

In [4]:
text = "The boy is 12 years old"

### Extract BOTH `12 years old` and `8 years old`

In [5]:
text = "The boy is 12 years old. The girl is 8 years old."

### Capture 2 groups: `quantity` and `date_type`
Iterate over the different groups and print them out.

In [6]:
text = "The boy is 12 years old. The girl is 96 months old."

## Some helpful tips!!

* `.*` -- `.` matches any character, while `*` means zero or more, and is "greedy". This is a catch all for "everything"
* `+` -- One or more
* `{1}` or `{1, 2}` -- Specify exactly how many times, or a range of times
* `?` -- Once or none
* `( ... )` -- Is a capture group
* `(?: ... )` -- A group.. But no capture
* `|` -- _or_ operator
* `^` -- _not_, or the start of a line
* `$` -- the end of a line

### Lets play a game!!
Acording to [this article](https://www.mattcutts.com/blog/seo-glossary-url-definitions/), the url is made up of many parts..
Lets create a regular expression that appropriately captures the different parts. 

### URL to test against

In [7]:
url = "http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s"

Part definitions: 

* The `protocol` is http. Other protocols include https, ftp, etc.
* The `host` or hostname is video.google.co.uk.
* The `subdomain` is video.
* The `domain` name is google.co.uk.
* The `top-level domain` or TLD is uk. The uk domain is also referred to as a country-code top-level domain or ccTLD. For google.com, the TLD would be com.
* The `second-level domain` (SLD) is co.uk.
* The `port` is 80, which is the default port for web servers. Other ports are possible; a web server can listen on port 8000, for example. When the port is 80, most people leave out the port.
* The `path` is /videoplay. Path typically refers to a file or location on the web server, e.g. /directory/file.html
* This URL has `parameters`. The name of one parameter is docid and the value of that parameter is -7246927612831078230. URLs can have lots parameters. Parameters start with a question mark _(?)_ and are separated with an ampersand _(&)_.
* See the “#00h02m30s”? That’s called a `fragment` or a named anchor. The Googlers I’ve talked to are split right down the middle on which way to refer it. Disputes on what to call it can be settled with arm wrestling, dance-offs, or drinking contests. 🙂 Typically the fragment is used to refer to an internal section within a web document. In this case, the named anchor means “skip to 2 minutes and 30 seconds into the video.” I think right now Google standardizes urls by removing any fragments from the url.

So.. 
We are tryign to capture 10 different groups. 
Let's try breaking this down.

Based on the definitions above.. 
We have some good guidlines to write our expressions.

### RULES

> Based on the definitions above.. 
Create a _SINGLE_ regular expression that captures _ALL_ of the different groups separately.
I have created an expression for `protocol`. 
Continue to build off of this until you have them all completed.

In [8]:
# start at the beginning.. and capture the http only.
# Wrapping it in () specifies that it is a group!
pattern = r"""

    ^                      # Specifies the beginning of the string
    (?P<protocol>http)     # For now.. Just say the literal match!

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'


Next.. 
We are moving into into the host.. 
Which is basically everything after the `http://` and before the port `:80`

In [9]:
pattern = r"""

    ^                        
    (?P<protocol>http)       
    ://                      # the part following the protocol
    (?P<host>.*)             # the group we want.. `.*` Captures everything
    :                        # specifies the end of this section. No group

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol',
        'host'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'


Subdomain!
This one is a little tricky, because the subdomain is a part of the host.
We will need to add a group.. Within the group!

In [10]:
pattern = r"""

    ^                      
    (?P<protocol>http)     
    ://                    
    (?P<host>              # specify the start of a group.. The host!
        (?P<subdomain>\w+) # specify a new group, subdomain.. `+` means greedy!
        \.                 # says there is a period
        .*                 # everything else
    )                      # end host group
    :                      

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'


Domain! Again.. This is a nested group. Should be pretty straightforward though!

In [11]:
pattern = r"""

    ^                       
    (?P<protocol>http)      
    ://                     
    (?P<host>               
        (?P<subdomain>\w+)  
        \.                  
        (?P<domain>.*)       # THIS IS WHAT CHANGED!! Easy peasy 
    )                        
    :                        

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'


top-level domain. 
We've been doing this all along!

In [12]:
pattern = r"""

    ^                                   
    (?P<protocol>http)                  
    ://                                 
    (?P<host>                           
        (?P<subdomain>\w+)              
        \.                              
        (?P<domain>                     # Start of the domain group!
            \w+\.\w+\.                  # There is a word.. period.. word.. period..
            (?P<top_level_domain>\w+)   # and our top-level domain!
        )                               # Close domain
    )                                   # Close host
    :                                   

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain', 
        'top_level_domain'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'
top_level_domain:    'uk'


Second level domain.. 
Yet another nest!

In [13]:
pattern = r"""

    ^                                       
    (?P<protocol>http)                      
    ://                                     
    (?P<host>                               
        (?P<subdomain>\w+)                  
        \.                                  
        (?P<domain>                         
            \w+\.                           # `google.` in this case
            (?P<second_level_domain>        # Start of the second level domain
                \w+\.                       # `co.`
                (?P<top_level_domain>\w+)   # and our top-level domain!
            )                               # Close second-level domain
        )                                   
    )                                       
    :                                       

"""

print(f"{'Target:': <20} '{url}'\n")
print('Notice the flip in order between top and second level domains!\n')
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain', 
        'second_level_domain',
        'top_level_domain'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

Notice the flip in order between top and second level domains!

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'
second_level_domain: 'co.uk'
top_level_domain:    'uk'


Finally we can get out of those nests!! The port should be easy :) 

In [14]:
pattern = r"""

    ^                                       
    (?P<protocol>http)                      
    ://                                     
    (?P<host>                               
        (?P<subdomain>\w+)                  
        \.                                  
        (?P<domain>                         
            \w+\.                           
            (?P<second_level_domain>        
                \w+\.                       
                (?P<top_level_domain>\w+)   
            )                               
        )                                   
    )                                       
    :                                       
    (?P<port>\d+)                           # Port group!

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain', 
        'second_level_domain',
        'top_level_domain',
        'port'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'
second_level_domain: 'co.uk'
top_level_domain:    'uk'
port:                '80'


In [15]:
pattern = r"""

    ^                                       
    (?P<protocol>http)                      
    ://                                     
    (?P<host>                               
        (?P<subdomain>\w+)                  
        \.                                  
        (?P<domain>                         
            \w+\.                           
            (?P<second_level_domain>        
                \w+\.                       
                (?P<top_level_domain>\w+)   
            )                               
        )                                   
    )                                       
    :                                       
    (?P<port>\d+)                           
    (?P<path>/\w+)                          # Path group!

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain', 
        'second_level_domain',
        'top_level_domain',
        'port',
        'path'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'
second_level_domain: 'co.uk'
top_level_domain:    'uk'
port:                '80'
path:                '/videoplay'


Parameters! These can be tricky since there can be one, many, or none of them. 

In [16]:
pattern = r"""

    ^                                       
    (?P<protocol>http)                      
    ://                                     
    (?P<host>                               
        (?P<subdomain>\w+)                  
        \.                                  
        (?P<domain>                         
            \w+\.                           
            (?P<second_level_domain>        
                \w+\.                       
                (?P<top_level_domain>\w+)   
            )                               
        )                                   
    )                                       
    :                                       
    (?P<port>\d+)                           
    (?P<path>/\w+)                          
    (?P<parameters>\?[a-zA-Z0-9,&=-]+)?     # Paramenters can include any of these characters.. and it's optional

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain', 
        'second_level_domain',
        'top_level_domain',
        'port',
        'path',
        'parameters'])

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'
second_level_domain: 'co.uk'
top_level_domain:    'uk'
port:                '80'
path:                '/videoplay'
parameters:          '?docid=-7246927612831078230&hl=en'


In [17]:
pattern = r"""

    ^                                       
    (?P<protocol>http)                      
    ://                                     
    (?P<host>                               
        (?P<subdomain>\w+)                  
        \.                                  
        (?P<domain>                         
            \w+\.                           
            (?P<second_level_domain>        
                \w+\.                       
                (?P<top_level_domain>\w+)   
            )                               
        )                                   
    )                                       
    :                                       
    (?P<port>\d+)                           
    (?P<path>/\w+)                          
    (?P<parameters>\?[a-zA-Z0-9,&=-]+)?     
    (?P<fragment>.*)                        # fragment is just a catch all for everything else!
    $          

"""

print(f"{'Target:': <20} '{url}'\n")
check_matches(
    pattern, 
    url, 
    [
        'protocol', 
        'host', 
        'subdomain', 
        'domain', 
        'second_level_domain',
        'top_level_domain',
        'port',
        'path',
        'parameters',
        'fragment'])                             

Target:              'http://video.google.co.uk:80/videoplay?docid=-7246927612831078230&hl=en#00h02m30s'

protocol:            'http'
host:                'video.google.co.uk'
subdomain:           'video'
domain:              'google.co.uk'
second_level_domain: 'co.uk'
top_level_domain:    'uk'
port:                '80'
path:                '/videoplay'
parameters:          '?docid=-7246927612831078230&hl=en'
fragment:            '#00h02m30s'


In [18]:
urls = """http://example.com/#basket
https://example.com/#attack
http://bath.example.net/#baseball
https://example.net/blood.php
https://www.example.com/
https://www.example.com/
http://babies.example.com/apparatus
https://www.example.com/behavior?baby=blood
https://www.example.com/brake/bat.php
http://www.example.com/birds/boat.aspx
http://www.example.com/boot
http://example.com/
https://example.edu/
http://authority.example.com/?back=apparatus&balance=basket
http://example.com/bridge/afternoon#believe
http://www.example.com/?approval=boy
http://example.com/beds.aspx
http://www.example.com/
https://example.net/
http://example.com/action.aspx
http://www.example.com/actor.php
http://example.com/blood/birds
http://example.com/base/attack.php
https://example.com/art.html
http://www.example.com/animal/border.php
http://bag.example.net/belief/beginner
http://www.example.com/
https://www.example.com/
http://www.example.com/arithmetic.aspx?authority=bubble
http://example.com/
http://www.example.com/?birds=board&border=basket#bait
http://example.com/bridge/advice
http://example.com/activity/branch?bead=attack&badge=box
http://example.net/
http://www.example.com/?breath=birthday
https://example.com/
http://example.net/beginner/arm.aspx
http://www.example.com/bead/boundary?agreement=adjustment&action=ball
https://blood.example.com/bird?attraction=box&amount=brick
https://example.com/bed#behavior
http://www.example.org/
https://example.com/?bone=bath&alarm=birthday
http://example.com/
http://example.com/
https://www.example.com/#actor
https://www.example.com/birthday.htm
http://www.example.org/
https://www.example.com/bone/belief.html
http://www.example.org/
http://example.net/?brick=board
http://www.example.com/?account=board&air=bee
https://birthday.example.net/
http://www.example.org/border
http://anger.example.com/
https://www.example.com/?book=act
http://example.net/?act=addition&anger=basket
https://branch.example.com/
http://www.example.com/art.html
https://www.example.net/
https://example.net/actor/afternoon.html
https://battle.example.com/
https://arithmetic.example.org/
http://example.com/
https://example.com/aftermath.html
http://breath.example.com/bite/brass.php
https://basket.example.com/#agreement
https://www.example.com/boat/animal
https://www.example.com/
http://www.example.org/
https://box.example.com/action.php#bell
https://example.net/
http://www.example.com/
https://www.example.com/bag.aspx
https://www.example.com/
https://example.com/appliance/brick.html
https://activity.example.com/
http://example.net/boundary/bite
http://www.example.org/boy/activity.html
http://www.example.com/
https://agreement.example.com/
http://example.edu/bed/attraction.html
http://www.example.com/
http://www.example.com/boot/activity
https://basket.example.com/
https://www.example.com/approval/advice
http://example.com/back.php
https://example.org/
https://advertisement.example.net/babies.htm#berry
https://www.example.net/?arithmetic=brass&boy=actor
https://authority.example.com/airplane.aspx
http://www.example.org/
http://example.edu/#act
http://www.example.com/acoustics/books
http://www.example.com/beds.php
https://www.example.com/?bag=birth&attraction=authority
http://www.example.com/
http://brick.example.com/appliance/boundary.html?bear=ants&agreement=bird
http://www.example.com/amusement?bag=bird&animal=agreement
http://example.org/bead/art.aspx
http://www.example.com/bedroom
http://example.com/
https://example.com/attack/bite.html?air=boy#acoustics
https://www.example.com/?breath=afternoon&blow=boundary
https://example.org/
http://www.example.com/bag/argument.php
http://www.example.com/arithmetic.php
https://example.net/angle/battle.php?breath=achiever
http://www.example.com/basketball
http://www.example.com/authority
http://example.com/bedroom.php
https://bee.example.com/bead/belief.html
http://boundary.example.edu/anger.php
https://www.example.net/animal#amusement
https://example.com/back/bomb.htm
http://www.example.com/
https://example.org/afternoon/brass?birthday=birth&arch=achiever
http://example.com/breath.php
https://www.example.com/account.php?bottle=bikes&behavior=account
https://www.example.com/
https://example.com/arch/action.htm
http://example.edu/
https://example.com/ball.aspx
http://www.example.net/
https://bath.example.net/arm/amusement
https://www.example.com/ball?basketball=airplane
http://apparatus.example.edu/approval/bit.aspx
https://example.com/air.php#baby
https://books.example.com/brick/bath.htm?act=apparatus
http://www.example.com/
http://example.com/?achiever=bag&airport=base
http://example.net/airplane.php?beds=bottle&bird=bomb#bath
http://example.net/afternoon.php
http://www.example.com/
https://www.example.com/arch/amusement
http://www.example.com/
http://bell.example.com/?blade=authority&alarm=birds
http://example.com/
http://www.example.net/?afternoon=bottle
http://www.example.edu/?babies=achiever
https://www.example.com/amount/book
https://www.example.net/brick/aftermath?air=attack
https://www.example.com/bath.php
https://www.example.com/brass.aspx
https://example.org/
http://www.example.com/
http://www.example.net/
https://www.example.org/
https://www.example.com/
http://amount.example.com/bead.php
http://example.org/arithmetic.php
https://www.example.com/
http://alarm.example.com/
http://example.com/book.php
https://example.com/bells/air.aspx
http://example.org/beef.php?basket=bat
https://example.com/#appliance
https://www.example.com/?basin=amount
https://bikes.example.com/action
http://www.example.com/birthday#brother
https://example.com/bells/battle.html
https://argument.example.com/agreement
http://example.org/addition/advice
https://example.edu/bead.aspx
http://example.com/baby/base
http://www.example.com/?back=breath&army=bear
https://example.com/amount/boat.php
https://www.example.edu/beef
https://www.example.com/
http://ants.example.com/beginner/action.html
https://example.net/bell/balance.html?attack=badge&advice=arm
https://example.com/
http://example.com/?beef=berry&blade=bit
https://www.example.com/ants.php
http://beds.example.com/birth#acoustics
http://www.example.com/#bag
http://bear.example.net/
http://berry.example.net/blood?books=basketball&brother=basin#bed
http://www.example.net/boy.php
https://www.example.edu/aunt.aspx
https://www.example.com/advertisement/boy
https://www.example.com/approval.html
https://example.com/
https://brick.example.com/boot
https://www.example.com/
https://www.example.net/box
https://example.com/bells.php
https://www.example.org/basketball
http://www.example.com/bite.php
http://example.net/beds.html
http://www.example.com/bike.aspx?border=balance#attack
https://example.com/balance/amusement.aspx
https://brass.example.com/
http://example.com/art/authority
https://example.com/
https://example.net/
https://bee.example.com/
http://apparel.example.com/
http://bridge.example.com/argument/bird
https://www.example.com/beginner.htm?bear=appliance
http://example.com/
https://example.com/blow.html
http://www.example.edu/advice
http://example.org/
http://www.example.com/
https://www.example.org/achiever/birds.html?advice=attraction
http://example.com/action/breath
http://www.example.com/
https://www.example.com/birds/bite
https://example.com/bells/birthday.html
http://baseball.example.net/?back=aunt&bag=bone
https://example.com/?branch=ball
https://example.com/
http://www.example.net/
http://example.com/boy/bead
https://www.example.net/belief
http://www.example.org/
https://example.com/#back
https://amusement.example.org/
https://example.com/?adjustment=amusement
https://www.example.com/birthday/air?alarm=attack&basketball=babies
https://www.example.net/
http://example.net/bath/activity
https://www.example.com/bikes.html
http://example.com/birds.aspx
http://attack.example.com/bee.html
http://www.example.com/
http://example.com/
http://example.com/apparatus?bead=behavior&breath=attack
https://example.com/?angle=action&ball=belief
http://www.example.com/beds/army.html
http://airport.example.net/
https://bikes.example.net/
https://beef.example.com/#belief
https://example.com/amount.aspx#basket
https://www.example.com/?approval=air&blood=brake
https://example.com/beds?bottle=airport&bird=branch
http://www.example.com/
http://example.com/beds.htm?addition=balance
http://www.example.net/ants/blood
http://www.example.org/airport?activity=argument&branch=birds
http://baseball.example.com/breath/back?amusement=achiever&anger=books
https://www.example.org/boot#amusement
http://example.com/
https://example.com/brass.php?bear=bike
http://www.example.com/#baseball
https://www.example.com/
https://www.example.com/
https://example.com/#breath
http://example.com/
https://afterthought.example.com/?boundary=birthday
http://www.example.com/beginner
http://www.example.com/birthday
https://www.example.org/
http://www.example.net/
http://example.org/afternoon.html#agreement
http://www.example.com/
https://example.net/amusement/berry.html?afterthought=bead
http://www.example.com/breath/believe
http://www.example.edu/airplane
http://www.example.com/
http://www.example.com/
https://www.example.com/#bear
http://www.example.net/
https://baseball.example.com/
http://actor.example.com/
http://example.org/?arch=brick
http://example.net/
http://www.example.net/
http://www.example.com/#actor
https://www.example.edu/
http://www.example.com/berry/brick.php
https://example.org/
https://art.example.com/authority
https://www.example.edu/bath/bath?attraction=breath
https://www.example.com/afterthought
https://www.example.com/
http://www.example.com/?argument=action#books
http://www.example.com/account
http://example.com/authority
http://www.example.com/board/brass.html?bomb=angle&airport=bedroom
https://www.example.com/amount/amount.php
https://battle.example.org/?appliance=bedroom
http://example.org/?blood=attack
https://actor.example.com/
http://example.com/
http://www.example.com/bone/bath.html
http://box.example.com/?appliance=birthday&bear=breath
http://example.com/
http://www.example.com/afterthought/birds
http://www.example.com/?ball=airport&book=attack
http://www.example.org/anger/ball
https://www.example.com/?bait=army&birth=blade
http://www.example.com/
https://www.example.com/?animal=adjustment&boy=board
http://www.example.net/
https://www.example.net/bite#art
http://example.edu/achiever.php
http://www.example.com/books.php?account=action&art=basin
http://example.com/?beef=branch&aftermath=afterthought
http://www.example.org/blade.aspx
http://www.example.edu/blow
http://example.net/
https://www.example.com/?blood=actor
https://www.example.com/?air=branch&authority=bait
https://example.com/appliance
https://example.edu/arch/advice?aftermath=acoustics
https://www.example.com/
https://bells.example.com/boundary?breath=acoustics&airplane=airplane#apparatus
https://bridge.example.net/brother/alarm.aspx?bee=angle
https://www.example.com/badge
http://birthday.example.com/#beef
https://www.example.com/
https://www.example.com/
http://www.example.com/bait.php#argument
http://example.net/
https://www.example.org/bee/bridge.htm
http://attraction.example.com/
http://www.example.com/?air=agreement
https://www.example.edu/brick/airport.aspx
http://www.example.com/?arithmetic=animal
https://www.example.com/basin?blade=behavior&basin=animal
http://example.com/
https://bike.example.net/ball/anger
https://www.example.com/bird/authority.aspx
http://www.example.com/
http://animal.example.com/airplane/bone.html?baby=apparel
http://www.example.com/
https://www.example.net/
http://www.example.com/
http://bottle.example.org/
http://www.example.com/
https://www.example.org/
https://www.example.com/boy?branch=berry
http://www.example.com/
https://amount.example.net/believe/badge#action
https://www.example.com/basket/anger
http://battle.example.com/
http://example.net/bat.aspx?bear=bite
https://www.example.com/
http://example.com/#argument
http://www.example.com/bottle/back
https://boundary.example.com/apparel.php
https://www.example.org/birthday.php
https://www.example.com/boot?army=berry&box=bag
http://www.example.com/bead/battle?back=alarm
https://www.example.com/
http://www.example.org/
http://example.org/boundary/adjustment.html
https://www.example.net/board/baseball.php#bit
https://www.example.com/bit
http://www.example.net/believe.php?bubble=bottle&blow=aunt
https://example.com/boat/approval
https://www.example.com/air.aspx
https://www.example.com/
http://www.example.com/
https://activity.example.com/agreement#bite
https://brick.example.net/
http://example.com/
http://example.com/action/basketball#bag
http://www.example.net/bike/bottle.html
http://www.example.com/
http://www.example.com/amount.php
https://www.example.com/
http://www.example.com/
http://example.net/
http://www.example.com/boot.aspx
http://www.example.com/afternoon.php
http://acoustics.example.edu/boat/amount
https://www.example.com/bone.html
https://www.example.net/back?balance=belief&base=appliance
https://www.example.org/blow/bells.php
https://www.example.com/acoustics/believe.aspx
http://brick.example.com/birth/airport.aspx
https://army.example.com/?boundary=airplane&blood=board
https://www.example.com/attack
http://www.example.com/berry/basketball.html
http://example.com/
https://example.com/brass
http://example.net/birthday.aspx
http://www.example.com/aftermath.htm?beef=boat&back=aunt
https://babies.example.com/
http://www.example.com/
http://www.example.com/
https://alarm.example.com/
http://www.example.com/
http://www.example.com/believe
https://www.example.com/achiever
http://www.example.net/
http://www.example.com/adjustment/birthday?attack=boat
http://www.example.com/bridge?bubble=brass
https://www.example.edu/aftermath
http://www.example.com/bedroom.php
https://example.com/?ball=arm#berry
https://www.example.com/argument/agreement.php
https://www.example.net/
https://www.example.com/
https://example.org/basketball.html
http://www.example.com/
http://www.example.com/
https://www.example.com/apparatus.aspx?attraction=bells&bath=afternoon
http://www.example.com/box/baby
https://www.example.com/
http://www.example.com/?attraction=basket&bag=apparatus
https://bit.example.com/
https://www.example.com/
https://example.com/amount.php
http://www.example.com/advice
http://www.example.com/
http://bath.example.edu/
http://example.edu/beds#attraction
https://example.com/
http://example.com/
https://example.net/apparatus.aspx
http://www.example.com/birthday.html
http://www.example.edu/attack.aspx
https://birth.example.com/badge
https://basketball.example.org/
https://birds.example.com/believe.html
https://www.example.com/arm.htm?base=believe
https://www.example.com/
https://example.com/?boundary=berry&bikes=bait
https://www.example.org/arch.htm
https://basketball.example.com/
http://www.example.com/
http://example.net/bead.php?air=birthday
https://www.example.com/
https://www.example.com/action.htm
http://www.example.com/agreement.html
https://www.example.com/art.htm
https://www.example.com/attack?bait=bottle
http://example.com/afternoon/advice.html
https://www.example.com/
https://www.example.com/
http://www.example.edu/
http://example.com/apparatus.php?box=account
https://www.example.org/birth/acoustics.php
https://bells.example.net/arithmetic/berry
http://www.example.com/birthday.html
https://example.com/#activity
https://www.example.net/?badge=advice&bead=argument
http://example.com/bikes
http://www.example.com/bait/army.php
http://www.example.com/
http://bait.example.com/baseball
https://www.example.edu/blood/basin
http://example.com/boy/airplane
http://example.net/
https://www.example.net/
http://example.com/bike.php
https://baby.example.com/brass.php?addition=bath&angle=activity#apparatus
https://www.example.com/advertisement?boot=back
http://attack.example.com/
https://www.example.net/blood#beef
http://baby.example.com/
https://www.example.org/?airplane=belief&advertisement=art
http://bikes.example.com/?apparel=birth&branch=bell
https://border.example.com/approval.html?brother=believe
http://www.example.net/
https://www.example.com/beds?bead=airport&belief=afterthought
https://example.com/
https://www.example.com/
http://example.com/basketball/birds
http://www.example.com/amusement/afterthought.php?bird=arm&addition=addition
http://www.example.net/?art=baby
http://example.com/
http://border.example.com/
https://www.example.com/
http://www.example.com/baseball#afterthought
https://brother.example.org/authority/birth.php
https://bikes.example.com/?badge=books&birds=bag
https://bath.example.com/bat/blood.html
http://boundary.example.com/
https://www.example.net/
http://www.example.com/boot
https://example.com/
https://example.org/act?blade=basin&bath=anger
http://example.com/bag
https://airport.example.com/
http://www.example.com/
https://bedroom.example.com/
https://www.example.com/?behavior=believe&bone=action
https://afterthought.example.com/addition.html#brother
http://www.example.com/bridge
http://www.example.com/bottle.html
https://blow.example.com/
https://basket.example.com/
http://www.example.com/achiever/agreement.php
https://addition.example.com/
http://advertisement.example.com/
http://bed.example.net/
http://www.example.org/
https://www.example.com/
https://example.org/
https://www.example.com/aftermath
https://www.example.com/
http://www.example.com/brick
http://www.example.net/blade.html?attack=angle
https://www.example.com/
https://www.example.com/
https://example.org/arithmetic.aspx"""

In [23]:
pattern = r"""

    ^
    (?P<protocol>http)
    ://
    (?P<host>
        (?P<subdomain>\w+)
        \.
        (?P<domain>
            \w+\.
            (?P<second_level_domain>
                \w+\.
                (?P<top_level_domain>\w+)
            )
        )
    )
    :?
    (?P<port>\d+)?
    (?P<path>/\w+)
    (?P<parameters>\?[a-zA-Z0-9,&=-]+)?
    (?P<fragment>.*)
    $          

"""

for url in urls.split('\n'):
    print(f"{'Target:': <20} '{url}'\n")
    check_matches(
        pattern, 
        url, 
        [
            'protocol', 
            'host', 
            'subdomain', 
            'domain', 
            'second_level_domain',
            'top_level_domain',
            'port',
            'path',
            'parameters',
            'fragment'])                             

Target:              'http://example.com/#basket'



MissingGroupError: No groups found in match! Try again